In [2]:
import geopandas as gpd

In [12]:
counties = gpd.read_file("../datasets/WA_County_Boundaries.geojson")
pop_centers = gpd.read_file("../datasets/b2_coverage_pop_centers_reduced_precision.geojson")

In [13]:
def find_intersecting_county(polygon, counties_gdf):
    intersecting_counties = counties_gdf[counties_gdf.geometry.contains(polygon)]
    # select counties that intersect the polygon with non-zero area (exclude touching-only)
    mask = counties_gdf.geometry.intersects(polygon) & ~counties_gdf.geometry.touches(polygon)
    intersecting_counties = counties_gdf[mask]
    poly_area = polygon.area
    intersecting_counties = intersecting_counties.copy()

    def _info(row):
        inter = row.geometry.intersection(polygon)
        inter_area = 0.0 if inter.is_empty else inter.area
        pct = (inter_area / poly_area * 100.0) if poly_area > 0 else None
        return {
            "JURISDICT_LABEL_NM": row["JURISDICT_LABEL_NM"],
            "Intersection_Area": inter_area,
            "Pct_of_Polygon": pct,
        }

    # replace the name column with a dict containing name, intersection area, and pct
    intersecting_counties["JURISDICT_LABEL_NM"] = intersecting_counties.apply(_info, axis=1)
    
    # remove entries whose Pct_of_Polygon is less than 0.9 (treat None as 0)
    pct_series = intersecting_counties["JURISDICT_LABEL_NM"].apply(
        lambda d: (d.get("Pct_of_Polygon") if isinstance(d, dict) and d.get("Pct_of_Polygon") is not None else 0.0)
    )
    intersecting_counties = intersecting_counties[pct_series >= 0.9]

    # optional: sort so largest intersections come first
    # intersecting_counties = intersecting_counties.sort_values(by="Pct_of_Polygon", ascending=False)
    return intersecting_counties['JURISDICT_LABEL_NM'].tolist()

In [14]:
pop_centers['Intersecting_Counties'] = pop_centers.geometry.apply(lambda geom: find_intersecting_county(geom, counties))

In [15]:
pop_centers['Counties_Count'] = pop_centers['Intersecting_Counties'].apply(len)

In [16]:
pop_centers['Counties_Count'].value_counts()


Counties_Count
1    454
2      4
Name: count, dtype: int64

In [17]:
pop_centers[['PlaceName', 'Counties_Count']][pop_centers['Counties_Count'] > 1]

,PlaceName,Counties_Count
86,Clear Lake CDP,2
141,Fairwood CDP,2
327,Queets CDP,2
452,Woodland UGA,2


In [19]:
pop_centers[['OBJECTID','PlaceName','Intersecting_Counties']]

,OBJECTID,PlaceName,Intersecting_Counties
0,2,Aberdeen Gardens CDP,"[{'JURISDICT_LABEL_NM': 'Grays Harbor', 'Inter..."
1,3,Acme CDP,"[{'JURISDICT_LABEL_NM': 'Whatcom', 'Intersecti..."
2,4,Addy CDP,"[{'JURISDICT_LABEL_NM': 'Stevens', 'Intersecti..."
3,5,Addy UGA,"[{'JURISDICT_LABEL_NM': 'Stevens', 'Intersecti..."
4,6,Ahtanum CDP,"[{'JURISDICT_LABEL_NM': 'Yakima', 'Intersectio..."
...,...,...,...
453,831,Woods Creek CDP,"[{'JURISDICT_LABEL_NM': 'Snohomish', 'Intersec..."
454,836,Yakima UGA,"[{'JURISDICT_LABEL_NM': 'Yakima', 'Intersectio..."
455,837,Yarrow Point,"[{'JURISDICT_LABEL_NM': 'King', 'Intersection_..."
456,840,Yelm UGA,"[{'JURISDICT_LABEL_NM': 'Thurston', 'Intersect..."


In [20]:
pop_centers.columns

Index(['OBJECTID', 'PlaceName', 'PlaceType', 'OnHighwayNetwork', 'ShapeSTArea',
       'ShapeSTLength', 'coverage_category', 'area_sq_km',
       'intersection_area_sq_km', 'area_diff_sq_km', 'status',
       'covers_osm_count', 'geometry', 'Intersecting_Counties',
       'Counties_Count'],
      dtype='object')

In [29]:
def get_max_intersecting_county(row):
    if len(row) == 1:
        first_county = row[0]
        return first_county['JURISDICT_LABEL_NM']
    else:
        max_intersection_area = 0
        max_row = None
        for intersecting_county in row:
            pct = intersecting_county['Pct_of_Polygon']
            if pct > max_intersection_area:
                max_row = intersecting_county
                max_intersection_area = pct
        return max_row['JURISDICT_LABEL_NM']
        
    return ''
pop_centers['County'] = pop_centers['Intersecting_Counties'].apply(get_max_intersecting_county)

In [31]:
# group by County Names
pop_centers['County'].value_counts().to_csv('pop_centers_county.csv')

In [ ]:
pop_centers[['County','PlaceName','OBJECTID']].to_csv('pop_centers_county.csv', index=False)

,County,PlaceName,OBJECTID
0,Grays Harbor,Aberdeen Gardens CDP,2
1,Whatcom,Acme CDP,3
2,Stevens,Addy CDP,4
3,Stevens,Addy UGA,5
4,Yakima,Ahtanum CDP,6
...,...,...,...
453,Snohomish,Woods Creek CDP,831
454,Yakima,Yakima UGA,836
455,King,Yarrow Point,837
456,Thurston,Yelm UGA,840
